<a href="https://colab.research.google.com/github/RCortez25/Scientific-Machine-Learning/blob/main/Differential_equations/SIR(NODE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

# Code walkthrough

In [ ]:
# A
using ComponentArrays, Lux, DiffEqFlux, OrdinaryDiffEq, Optimization, OptimizationOptimJL,
    OptimizationOptimisers, Random, Plots, ModelingToolkit

# B
random_number_generator = Random.default_rng()
Random.seed!(rng, 42)

# C
#------ Generate ground-truth data
@parameters t β γ N
@variables S(t) I(t) R(t)
Dt = Differential(t)

eqs = [
    Dt(S) ~ -(β*S*I)/N,
    Dt(I) ~ ((β*S*I)/N) - γ*I,
    Dt(R) ~ γ*I
]

@named system = ODESystem(eqs, t, [S, I, R], [β, γ, N])
simplified = structural_simplify(system)

parameter_map = Dict(β => 0.3, γ => 0.1, N => 1000)
initial_conditions = Dict(I => 1, R => 0, S => 1000 - 1 - 0)
timespan = (0.0, 160.0) # in days

problem = ODEProblem(simplified,
                     merge(initial_conditions, parameter_map),
                     timespan)

solution = solve(problem, Tsit5(), saveat=0.5)
ground_truth = Array(solution)

**A** - Importing the packages
*   `ComponentArrays` is for packaging parameters into a single vector with names and structure
*   `DiffEqFlux` for using `NeuralODE`
*   `OptimizationOptimisers` adapter for `Optimisers.jl`, for using ADAM.
*   `Random` for seeding random number generators for reproducibility

**B** - Create a seeded random number generator for reproducibility

**C** - The rest of the code is used for solving the ODE system and generate ground-truth data for comparing with the NN results. `ground_truth` stores the results of the integrator.

In [ ]:
# A
const input_dimension = 4 # t, S, I, R
const output_dimension = 3 # Derivatives of S, I, R

# B
layer_0 = Lux.Dense(input_dimension, 32, Lux.tanh)
layer_1 = Lux.Dense(32, 32, Lux.tanh)
layer_2 = Lux.Dense(32, output_dimension)

# C
NN = Lux.Chain(
    layer_0,
    layer_1,
    layer_2
)

# D
parameters, state = Lux.setup(random_number_generator, NN)

# E
neural_ODE_problem = NeuralODE(NN, timespan, Tsit5(); saveat = 0.1)

**A** - Defining the input and output dimensions. `const` locks them up in order to avoid accidental re-writing of the dimensions. In the case of NeuralODEs, as one is predicting the derivative, the inputs are $t,S,I,R$ in that order, and the output numbers represent the value of their derivatives $\dot{S},\dot{I},\dot{R}$.

**B** - Definition of the NN architecture, in this case, 3 layers:
*   `layer_0` is the input layer, recieves 4 inputs and outputs 32 numbers (this is arbitrary and can be changed) using `tanh` activation function.
*   `layer_1` First hidden layer with 32 inputs (from the previous layer) and 32 outputs using `tanh` activation layer.
*   `layer_2` is the output layer. It recieves 32 inputs (from the previous layer) and outputs 3 numbers, namely, the derivatives as stated before.

**C** - Creating of the NN using the defined layers

**D** - Initializing the network. It returns two objects:
*   `parameters` which is the set of all trainable parameters and biases to be optimized later during training.
*   `state` all non-trainable internal states some layers keep (BatchNorm running means, etc).